In [11]:
import pandas as pd
import re
import requests
import json
import functools
import time
from pprint import pprint
from docx import Document
from os import listdir, environ
from os.path import isfile, join, abspath
from collections import defaultdict

In [3]:

datafiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]
columns = ['Дата', 'Адрес', 'Оксид углерода', 'Оксид азота', 'Диоксид азота', 'Диоксид серы', 'Озон', 'Взвешенные частицы PM10', 'Взвешенные частицы PM2.5']

In [4]:
def addDate(fn):
    df = pd.DataFrame(columns=columns)
    with open(abspath('data/'+fn), 'rb') as f:
        pollution_report = Document(f)

        adr_cnt = 0
        for i, paragraph in enumerate(pollution_report.paragraphs):
            row_df = dict.fromkeys(columns)
            adress = re.findall("адресу: (.*?)\\(станции", paragraph.text)
            if adress:
                row_df['Адрес'] = adress[0]
                row_df['Дата']  = fn.split('.')[0]
                for j in range(1, len(pollution_report.tables[adr_cnt].rows)):
                    row = pollution_report.tables[adr_cnt].rows[j]
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text)
                    row_df[row_text[0]] = row_text[1]
                df = df.append(row_df, ignore_index=True)
                adr_cnt += 1
    return df

In [33]:
# Construct dataframe from all of the documents
df = pd.DataFrame(columns=columns)

for file in datafiles:
    df = df.append(addDate(file), ignore_index=True)

In [34]:
# Filter out adresses for better mapbox search
def filter_adress(adr):
    expanded = functools.reduce(lambda x, y: x.replace(y, abbr[y]), abbr, adr).rstrip() 
    tokens = expanded.split(' ')
    if ('город' not in tokens and 'поселок' not in tokens):
        expanded = 'Санкт-Петербург, ' + expanded
    expanded = 'Россия, ' + expanded
    return expanded

abbr = {'пр.':'проспект', 'ул.': 'улица', 'В.О.': '', 'пер.': 'переулок', 'г.': 'город', 'пос.': 'поселок', 'М.': 'Максима'}
df['Адрес'] = df['Адрес'].map(filter_adress)
adresses = df['Адрес'].unique()

print(adresses)
# Append country, city name to the adresses
for i in range(len(adresses)):
    pass


['Россия, Санкт-Петербург, Московский проспект, дом 19'
 'Россия, Санкт-Петербург, Средний проспект, дом 74'
 'Россия, Санкт-Петербург, проспект КИМа, дом 26 лит. А'
 'Россия, Санкт-Петербург, улица Карбышева, дом 7'
 'Россия, Санкт-Петербург, улица Ольги Форш, дом 6'
 'Россия, Санкт-Петербург, проспект Маршала Жукова, дом 30, корп. 3'
 'Россия, город Колпино, Красная улица, дом 1а'
 'Россия, поселок Металлострой, Железнодорожная улица, дом 13'
 'Россия, Санкт-Петербург, Малоохтинский проспект, дом 98'
 'Россия, Санкт-Петербург, Индустриальный проспект, д.64'
 'Россия, Санкт-Петербург, Уткин проспект, д.16'
 'Россия, Санкт-Петербург, Волхонское шоссе, д.116, корп. 3'
 'Россия, город Кронштадт, улица Ильмянинова, дом 4'
 'Россия, город Сестрорецк, улица Максима Горького, дом 2'
 'Россия, Санкт-Петербург, улица Севастьянова, дом 11'
 'Россия, Санкт-Петербург, улица Тельмана, дом 24'
 'Россия, Санкт-Петербург, улица Профессора Попова, дом 48'
 'Россия, город Ломоносов, улица Федюнинского,

In [28]:
# Get geo-coordinates for each unique adress
#bbox = '29.5313, 59.6691, 30.8386, 60.2466'
#access_token = 'pk.eyJ1IjoiYWxleDQ5MiIsImEiOiJjank1b3BleXcwOHM4M29tbHY1bWVhaXA1In0.HY6gliRl51T_WMSxdOGbJw'
#base_url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?bbox={}&access_token={}"
acess_token = "126669782449696e15901653x3116"
base_url = 'https://geocode.xyz/{}?json=1&auth={}'
georef = dict.fromkeys(adresses)
    
for adress in adresses:

    search_result = requests.get(base_url.format(adress, acess_token), timeout=100).json()
    pprint(search_result)
    georef.update({adress: [search_result['longt'], search_result['latt']]})
    

{'alt': {},
 'elevation': {},
 'latt': '59.87203',
 'longt': '30.31976',
 'remaining_credits': '-32',
 'standard': {'addresst': 'Московский Проспект',
              'city': 'Санкт-Петербург',
              'confidence': '0.4',
              'countryname': 'Russian Federation',
              'postal': {},
              'prov': 'RU'}}
{'alt': {},
 'elevation': {},
 'latt': '59.94592',
 'longt': '30.28547',
 'remaining_credits': '-33',
 'standard': {'addresst': 'Средний Проспект',
              'city': 'Санкт-Петербург',
              'confidence': '0.4',
              'countryname': 'Russian Federation',
              'postal': {},
              'prov': 'RU'}}
{'alt': {},
 'elevation': {},
 'latt': '59.95332',
 'longt': '30.24221',
 'remaining_credits': '-34',
 'standard': {'addresst': 'проспект КИМа',
              'city': 'Санкт-Петербург',
              'confidence': '0.9',
              'countryname': 'Russian Federation',
              'postal': {},
              'prov': 'RU',
     

In [35]:
georef

{'Россия, Санкт-Петербург, Московский проспект, дом 19': ['30.31976',
  '59.87203'],
 'Россия, Санкт-Петербург, Средний проспект, дом 74': ['30.28547', '59.94592'],
 'Россия, Санкт-Петербург, проспект КИМа, дом 26 лит. А': ['30.24221',
  '59.95332'],
 'Россия, Санкт-Петербург, улица Карбышева, дом 7': ['30.35872', '59.99687'],
 'Россия, Санкт-Петербург, улица Ольги Форш, дом 6': ['30.39148', '60.04297'],
 'Россия, Санкт-Петербург, проспект Маршала Жукова, дом 30, корп. 3': ['30.22986',
  '59.85633'],
 'Россия, город Колпино, Красная улица, дом 1а': ['30.60285', '59.74381'],
 'Россия, поселок Металлострой, Железнодорожная улица, дом 13': ['30.56708',
  '59.81286'],
 'Россия, Санкт-Петербург, Малоохтинский проспект, дом 98': ['30.40106',
  '59.93159'],
 'Россия, Санкт-Петербург, Индустриальный проспект, д.64': ['30.47274',
  '59.94882'],
 'Россия, Санкт-Петербург, Уткин проспект, д.16': ['30.42562', '59.93601'],
 'Россия, Санкт-Петербург, Волхонское шоссе, д.116, корп. 3': ['30.17530',
 

In [30]:
df['lon'] = df['Адрес'].apply(lambda row: georef[row][0])
df['lat'] = df['Адрес'].apply(lambda row: georef[row][1])

KeyError: 'Московский проспект, дом 19'

In [13]:
def numberIt(cell):
    if not cell or '*' in cell:
        return None
    tokens = cell.split(' ')
    if ('менее' in tokens):
        return float(tokens[-1])/2
    else:
        return float(cell)
    
df.iloc[:,2:9] = df.iloc[:,2:9].applymap(numberIt)

In [14]:
df.to_csv('spb_pollution_record.csv')